# Bayesian Networks in Python

I will build a Bayesian (Belief) Network for the Alarm example in the textbook using the Python library [pgmpy](https://pgmpy.org/).


In [138]:
%pip install -q pgmpy

Note: you may need to restart the kernel to use updated packages.


## Defining the  Bayesian Network

![The Alarm Bayes Network](Alarm_BN.png)

In [139]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork

model = DiscreteBayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "JohnCalls"),
        ("Alarm", "MaryCalls"),
    ]
)

# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.999], [0.001]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.998], [0.002]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_johncalls = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.95, 0.1], [0.05, 0.9]],
    evidence=["Alarm"],
    evidence_card=[2],
)
cpd_marycalls = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.99, 0.3], [0.01, 0.7]],
    evidence=["Alarm"],
    evidence_card=[2],
)

# Associating the parameters with the model structure
model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls
)

Let's check what variables are (conditionally) independent in the model.

In [140]:
model.get_independencies()

(Burglary ⟂ JohnCalls | Alarm)
(Burglary ⟂ MaryCalls | Alarm)
(Burglary ⟂ Earthquake)
(MaryCalls ⟂ Earthquake | Alarm)
(JohnCalls ⟂ MaryCalls | Alarm)
(JohnCalls ⟂ Earthquake | Alarm)

See: [pmgpy: Bayesian Networks](https://pgmpy.org/models/bayesiannetwork.html)

# Exact Inference

## Joint Probability

The following calculates $P(B=false, E=false, A=true, J=true, M=false)$,
$P(B)$, $P(E)$, and $P(B, E)$.

In [141]:
[model.get_state_probability({'Burglary': 0,
                              'Earthquake': 0,
                              "Alarm": 1,
                              "JohnCalls": 1,
                              "MaryCalls": 0}
                             ),
 model.get_state_probability({'Burglary': 1}),
 model.get_state_probability({'Earthquake': 1}),
 model.get_state_probability({'Burglary': 1, 'Earthquake': 1})]

[np.float64(0.00026919053999999995),
 np.float64(0.001),
 np.float64(0.0020000000000000005),
 np.float64(2e-06)]

In [142]:
# B and E are independent... P(B=false AND E=false) = P(B=false) * P(E=false)
0.001 * 0.002

2e-06

## Conditional Probabilities

We calculate $\mathbf{P}(Burglary | JohnCalls, MaryCalls)$, the probability that a burglary is happening if both neighbors call.

The exact method used here is called variable elimination and works since the model is so small.

In [143]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model) 

print(inference.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1}))

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


There is a 28% chance of burglary when both call.

We will estimate the same conditional probability with several approximate methods below.

# Approximate Inference

See: [pmgpy Approximate Inference Using Sampling](https://pgmpy.org/approx_infer/approx_infer.html)

We will call the sampling methods directly. A more convenient interface is provided as `model.simulate()`
and `inference.query()` which will automatically choose the correct sampling method and calculate probabilities from the 
samples.

In [144]:
from pgmpy.sampling import BayesianModelSampling
inference = BayesianModelSampling(model)

## Prior Sampling

Samples a complete event from the network.
An event is an assignment for each variable sampled from the network. Prior sampling is called `forward_sample` in `pgmpy`.

In [145]:
inference.forward_sample(size=10)

Generating for node: MaryCalls: 100%|██████████| 5/5 [00:00<00:00, 163.29it/s]


,Burglary,Alarm,Earthquake,JohnCalls,MaryCalls
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,0,0,0,0,0


Estimate probabilities from larger samples.

In [146]:
s = inference.forward_sample(size=1000000)
s.value_counts(normalize=True)

Generating for node: MaryCalls: 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]


Burglary  Alarm  Earthquake  JohnCalls  MaryCalls
0         0      0           0          0            0.936511
                             1          0            0.049639
                             0          1            0.009381
                 1           0          0            0.001308
          1      0           1          1            0.000594
1         1      0           1          1            0.000591
0         0      0           1          1            0.000521
          1      1           1          1            0.000366
                 0           1          0            0.000280
1         1      0           1          0            0.000238
0         1      1           1          0            0.000168
1         1      0           0          1            0.000070
0         0      1           1          0            0.000070
          1      0           0          1            0.000068
1         0      0           0          0            0.000060
0         1      1  

In [147]:
s["Burglary"].value_counts(normalize=True)

Burglary
0    0.999002
1    0.000998
Name: proportion, dtype: float64

## Rejection Sampling
Uses rejection sampling by ignoring the samples that are not consistent with the evidence.

I fix here the variables `JohnCalls` and `MaryCalls` to 1.

In [148]:
from pgmpy.factors.discrete import State
evidence = [State(var='JohnCalls', state=1), State(var='MaryCalls', state=1)]

In [149]:
inference.rejection_sample(evidence = evidence, size = 10)

100%|██████████| 10/10 [00:00<00:00, 17.40it/s]


,Burglary,Alarm,Earthquake,JohnCalls,MaryCalls
0,0,0,0,1,1
1,0,1,1,1,1
2,1,1,0,1,1
3,0,0,0,1,1
4,0,1,1,1,1
5,0,1,0,1,1
6,1,1,0,1,1
7,0,0,0,1,1
8,0,0,0,1,1
9,0,0,0,1,1


Estimate probabilities from larger samples. It looks like size is the number of not rejected samples, so I decrease the number.

In [163]:
s = inference.rejection_sample(evidence = evidence, size=1000)
s.value_counts(normalize=True)



















100%|██████████| 1000/1000 [00:01<00:00, 848.56it/s]


Burglary  Alarm  Earthquake  JohnCalls  MaryCalls
0         1      0           1          1            0.303
1         1      0           1          1            0.260
0         0      0           1          1            0.230
          1      1           1          1            0.206
1         1      1           1          1            0.001
Name: proportion, dtype: float64

In [ ]:
s['Burglary'].value_counts(normalize=True)

Burglary
0    0.739
1    0.261
Name: proportion, dtype: float64

## Importance Sampling
Next, we use importance sampling here.

In [152]:
inference.likelihood_weighted_sample(evidence = evidence, size = 10)

Generating for node: MaryCalls: 100%|██████████| 5/5 [00:00<00:00, 106.57it/s]


,Burglary,Alarm,Earthquake,JohnCalls,MaryCalls,_weight
0,0,0,0,1,1,0.0005
1,0,0,0,1,1,0.0005
2,0,0,0,1,1,0.0005
3,0,0,0,1,1,0.0005
4,0,0,0,1,1,0.0005
5,0,0,0,1,1,0.0005
6,0,0,0,1,1,0.0005
7,0,0,0,1,1,0.0005
8,0,0,0,1,1,0.0005
9,0,0,0,1,1,0.0005


To estimate probabilities, the sample weight has to be used (which I don't implement here).

## Gibbs Sampling

Looks like `pmgpy` does not implement Gibbs Sampling with evidence using this interface. See queries below.

In [153]:
from pgmpy.sampling import GibbsSampling
gibbs_chain = GibbsSampling(model)
gibbs_chain.sample(size=10)

100%|██████████| 9/9 [00:00<00:00, 581.07it/s]

,Burglary,Alarm,Earthquake,JohnCalls,MaryCalls
0,0,1,0,1,0
1,1,1,0,1,1
2,0,1,0,0,1
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,0,0,0,0,0


In [154]:
s = gibbs_chain.sample(size=1000)
s.value_counts(normalize=True)

100%|██████████| 999/999 [00:00<00:00, 1092.84it/s]


Burglary  Alarm  Earthquake  JohnCalls  MaryCalls
0         0      0           0          0            0.940
                             1          0            0.041
                             0          1            0.010
                 1           0          0            0.002
1         1      0           1          1            0.002
0         0      0           1          1            0.001
                 1           1          0            0.001
          1      0           1          0            0.001
                                        1            0.001
1         1      0           0          1            0.001
Name: proportion, dtype: float64

## Approximate Query

This queries the probability of a variable given the evidence. Most likely it uses Gibbs sampling here.

In [168]:
from pgmpy.inference import ApproxInference
inference = ApproxInference(model)

print(inference.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1}))









































100%|██████████| 10000/10000 [00:13<00:00, 760.18it/s]

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7273 |
+-------------+-----------------+
| Burglary(1) |          0.2727 |
+-------------+-----------------+



/home/mhahsler/github/CS7320-AI/.venv/lib/python3.12/site-packages/pgmpy/inference/ApproxInference.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  samples.groupby(variables).size() / samples.shape[0], state_names


# Learning Bayes Networks from Data

`pgmpy` provides a `fit` function to learn the model.